# Assignment 7

Train a Transformer model for Machine Translation from Russian to English.  
Dataset: http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz   
Make all source and target text to lower case.  
Use following tokenization for english:  
```
import sentencepiece as spm

...
spm.SentencePieceTrainer.Train('--input=data/text.en --model_prefix=bpe_en --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

tok_en = spm.SentencePieceProcessor()
tok_en.load('bpe_en.model')

TGT = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_en.encode_as_pieces(x),
    batch_first=True,
)

...
TGT.build_vocab(..., min_freq=5)
...

```
Score: corpus-bleu `nltk.translate.bleu_score.corpus_bleu`  
Use last 1000 sentences for model evalutation (test dataset).  
Use your target sequence tokenization for BLEU score.  
Use max_len=50 for sequence prediction.  


Hint: You may consider much smaller model, than shown in the example.  

Baselines:  
[4 point] BLEU = 0.05  
[6 point] BLEU = 0.10  
[9 point] BLEU = 0.15  

[1 point] Share weights between target embeddings and output dense layer. Notice, they have the same shape.


Readings:
1. BLUE score how to https://machinelearningmastery.com/calculate-bleu-score-for-text-python/
1. Transformer code and comments http://nlp.seas.harvard.edu/2018/04/03/attention.html

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!wget http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz
!tar -xzvf training-parallel-nc-v13.tgz
!mv training-parallel-nc-v13 data

--2020-03-01 19:23:47--  http://data.statmt.org/wmt18/translation-task/training-parallel-nc-v13.tgz
Resolving data.statmt.org (data.statmt.org)... 129.215.197.184
Connecting to data.statmt.org (data.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 113157482 (108M) [application/x-gzip]
Saving to: ‘training-parallel-nc-v13.tgz’

training-parallel-n 100%[===================>] 107.92M  24.7MB/s    in 5.1s    

2020-03-01 19:23:53 (21.2 MB/s) - ‘training-parallel-nc-v13.tgz’ saved [113157482/113157482]

training-parallel-nc-v13/
training-parallel-nc-v13/news-commentary-v13.ru-en.ru
training-parallel-nc-v13/news-commentary-v13.cs-en.en
training-parallel-nc-v13/news-commentary-v13.de-en.de
training-parallel-nc-v13/news-commentary-v13.ru-en.en
training-parallel-nc-v13/news-commentary-v13.zh-en.zh
training-parallel-nc-v13/news-commentary-v13.zh-en.en
training-parallel-nc-v13/news-commentary-v13.cs-en.cs
training-parallel-nc-v13/news-commentary

In [0]:
!pip install sentencepiece

     |████████████████████████████████| 1.0MB 3.4MB/s 


In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
import tqdm
from torchtext import datasets, data
import sentencepiece as spm


DEVICE = 'cuda'

In [0]:
!ls data

news-commentary-v13.cs-en.cs  news-commentary-v13.ru-en.en
news-commentary-v13.cs-en.en  news-commentary-v13.ru-en.ru
news-commentary-v13.de-en.de  news-commentary-v13.zh-en.en
news-commentary-v13.de-en.en  news-commentary-v13.zh-en.zh


In [0]:
# tokenize english 
with open('data/news-commentary-v13.ru-en.en') as f:
    with open('data/text.en', 'w') as out:
            out.write(f.read().lower())
        
spm.SentencePieceTrainer.Train('--input=data/text.en --model_prefix=bpe_en --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

True

In [0]:
# tokenize russian

with open('data/news-commentary-v13.ru-en.ru') as f:
    with open('data/text.ru', 'w') as out:
            out.write(f.read().lower())
        
spm.SentencePieceTrainer.Train('--input=data/text.ru --model_prefix=bpe_ru --vocab_size=32000 --character_coverage=0.98 --model_type=bpe')

True

In [0]:
tok_ru = spm.SentencePieceProcessor()
tok_ru.load('bpe_ru.model')

tok_en = spm.SentencePieceProcessor()
tok_en.load('bpe_en.model')

SRC = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_ru.encode_as_pieces(x),
    batch_first=True,
)

TGT = data.Field(
    fix_length=50,
    init_token='<s>',
    eos_token='</s>',
    lower=True,
    tokenize = lambda x: tok_en.encode_as_pieces(x),
    batch_first=True,
)

fields = (('src', SRC), ('tgt', TGT))

In [0]:
with open('data/text.ru') as f:
    src_snt = list(map(str.strip, f.readlines()))
    
with open('data/text.en') as f:
    tgt_snt = list(map(str.strip, f.readlines()))
    
examples = [data.Example.fromlist(x, fields) for x in tqdm.tqdm_notebook(zip(src_snt, tgt_snt))]
test = data.Dataset(examples[-1000:], fields)
train, valid = data.Dataset(examples[:-1000], fields).split(0.9)

In [0]:
print('src: ' + " ".join(train.examples[100].src))
print('tgt: ' + " ".join(train.examples[100].tgt))

src: ▁кроме ▁того , ▁глобальная ▁ ф инансовая ▁стабильность ▁в ▁настоящее ▁время ▁зависит ▁от ▁более ▁тесного ▁сотрудничества ▁на ▁международном ▁уровне , ▁с ▁ужесто чением ▁правил ▁на ▁национальном ▁уровне .
tgt: ▁moreover , ▁global ▁financial ▁stability ▁now ▁depends ▁upon ▁greater ▁cooperation ▁at ▁the ▁international ▁level , ▁with ▁tighter ▁enforcement ▁of ▁rules ▁at ▁the ▁national ▁level .


In [0]:
len(train), len(valid), len(test)

(210743, 23416, 1000)

In [0]:
TGT.build_vocab(train, min_freq=5)
SRC.build_vocab(train, min_freq=5)

In [0]:
!wget https://raw.githubusercontent.com/isikus/devnull/master/transformer2.py
!mv transformer2.py transformer.py

--2020-03-01 19:25:54--  https://raw.githubusercontent.com/isikus/devnull/master/transformer2.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9818 (9.6K) [text/plain]
Saving to: ‘transformer2.py’

transformer2.py     100%[===================>]   9.59K  --.-KB/s    in 0s      

2020-03-01 19:25:54 (170 MB/s) - ‘transformer2.py’ saved [9818/9818]



In [0]:
from transformer import make_model, Batch
from torch.autograd import Variable
    
class BucketIteratorWrapper(DataLoader):
    __initialized = False

    def __init__(self, iterator: data.Iterator):
        #super(BucketIteratorWrapper,self).__init__()
        self.batch_size = iterator.batch_size
        self.num_workers = 1
        self.collate_fn = None
        self.pin_memory = False
        self.drop_last = False
        self.timeout = 0
        self.worker_init_fn = None
        self.sampler = iterator
        self.batch_sampler = iterator
        self.__initialized = True

    def __iter__(self):
        return map(
            lambda batch: Batch(batch.src, batch.tgt, pad=TGT.vocab.stoi['<pad>']),
            self.batch_sampler.__iter__()
        )

    def __len__(self):
        return len(self.batch_sampler)
    
class MyCriterion(nn.Module):
    def __init__(self, size, padding_idx, smoothing=0.0):
        super(MyCriterion, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None
        
    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))

In [0]:
class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))
        
def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [0]:
class LossCompute:
    "A simple loss compute and train function."
    def __init__(self, generator, criterion, opt=None):
        self.generator = generator
        self.criterion = criterion
        self.opt = opt
        
    def __call__(self, x, y, norm):
        x = self.generator(x)
        loss = self.criterion(x.contiguous().view(-1, x.size(-1)), 
                              y.contiguous().view(-1)) / norm
        if self.opt is not None:
            loss.backward()
            self.opt.step()
            self.opt.optimizer.zero_grad()
        return loss.data.detach().item() * norm.data.detach().item()

In [0]:
torch.cuda.empty_cache()

batch_size = 64
num_epochs = 3

train_iter, valid_iter, test_iter = data.BucketIterator.splits((train, valid, test), 
                                              batch_sizes=(batch_size, batch_size, batch_size), 
                                  sort_key=lambda x: len(x.src),
                                  shuffle=True,
                                  device=DEVICE,
                                  sort_within_batch=False)
                                  
train_iter = BucketIteratorWrapper(train_iter)
valid_iter = BucketIteratorWrapper(valid_iter)
test_iter = BucketIteratorWrapper(test_iter)

model = make_model(len(SRC.vocab), len(TGT.vocab), N=6)
model = model.to(DEVICE)
criterion = MyCriterion(size=len(TGT.vocab), padding_idx=TGT.vocab.stoi['<pad>'], smoothing=0.1)
optimizer = NoamOpt(model.src_embed[0].d_model, 1, 2000,
        torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

/content/transformer.py:235: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:43: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


In [0]:
def train_epoch(data_iter, model, criterion, loss_compute):
    data_iter = tqdm.tqdm_notebook(data_iter)
    total_tokens = 0
    total_loss = 0
    for batch in data_iter:        
        pred = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(pred, batch.trg_y, batch.ntokens)
        
        total_loss += loss
        total_tokens += batch.ntokens.data.detach().item()
        data_iter.set_postfix(loss = loss / batch.ntokens.data.detach().item())
    return total_loss / total_tokens

def valid_epoch(data_iter, model, criterion, loss_compute):
    data_iter = tqdm.tqdm_notebook(data_iter)
    total_tokens = 0
    total_loss = 0
    for batch in data_iter:        
        pred = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(pred, batch.trg_y, batch.ntokens)
        
        total_loss += loss
        total_tokens += batch.ntokens.data.detach().item()
        data_iter.set_postfix(loss = loss / batch.ntokens.data.detach().item())

    return total_loss / total_tokens

for epoch in range(num_epochs):
    model.train()
    loss = train_epoch(train_iter, model, criterion,
                       LossCompute(model.generator, criterion, optimizer))
    print('train', loss)
    
    model.eval()
    with torch.no_grad():
        loss = valid_epoch(valid_iter, model, criterion,
                           LossCompute(model.generator, criterion, None))
        print('valid', loss)


train 4.308500647219745

train 4.308500647219745



valid 3.2844281442041168

valid 3.2844281442041168



train 2.9438094521943863

train 2.9438094521943863



valid 2.5831497068892038

valid 2.5831497068892038


In [0]:
torch.save(model.state_dict(), 'iwslt3.pt')

In [0]:
!ls -lAh iwslt3.pt

In [0]:
!cp iwslt3.pt /content/drive/My\ Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#!cp /content/drive/My\ Drive/iwslt.pt .
loss

2.319877185837412

In [0]:
hypotheses = []
references = []

model.eval()
with torch.no_grad():
    for batch in test_iter:
        pred = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        sents = torch.argmax(torch.softmax(pred, dim=-1), dim=-1)
        hypotheses.extend([[TGT.vocab.itos[ix] for ix in sent] for sent in sents])
        references.extend([[[TGT.vocab.itos[ix] for ix in sent]] for sent in batch.tgt_y])

In [0]:
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

In [0]:
corpus_bleu(
    references, hypotheses, smoothing_function=SmoothingFunction().method3,
    auto_reweigh=True
)

5.2540557287914e-05

In [0]:
from transformer import subsequent_mask

def beam_search(model, src, src_mask, max_len=10, k=5):
    memory = model.encode(src, src_mask)
    start_token = TGT.vocab.stoi["<s>"]
    end_token = TGT.vocab.stoi["</s>"]
    ys = torch.ones(1, 1).fill_(start_token).type_as(src.data)
    beam = [(ys, 0)]
    for i in range(max_len):
        candidates= []
        candidates_proba = []
        prev_prob = None
        for snt, snt_proba in beam:
            if snt[0][-1] == end_token:
                candidates.append(snt)
                candidates_proba.append(snt_proba)
            else:
                proba = model.decode(memory, src_mask, snt,
                                     subsequent_mask(snt.size(1)).type_as(src.data))
                proba = proba[0][i]
                best_k = torch.argsort(-proba)[:k].tolist()
                proba = proba.tolist()
                prev_prob = proba
                for tok in best_k:
                    candidates.append(torch.cat([snt, torch.ones(1, 1).type_as(src.data).fill_(tok)], dim=1))
                    candidates_proba.append(snt_proba + np.log(proba[tok])) 
         
        best_candidates = np.argsort(-np.array(candidates_proba))[:k]
        beam = [(candidates[j], candidates_proba[j]) for j in best_candidates]
    return beam

In [0]:
eos = '</s>'

In [0]:
type(model)

transformer.EncoderDecoder

In [0]:
model.eval()
with torch.no_grad():
    for i, batch in enumerate(valid_iter):
        src = batch.src[:1]
        src_key_padding_mask = src != TGT.vocab.stoi['<pad>']
        beam = beam_search(model, src, src_key_padding_mask, max_len=8, k=5)
        
        seq = []
        for i in range(1, src.size(1)):
            sym = SRC.vocab.itos[src[0, i]]
            if sym == eos: break
            seq.append(sym)
        seq = tok_ru.decode_pieces(seq)
        print("\nSource:", seq)
        
        print("Translation:")
        for pred, pred_proba in beam:                
            seq = []
            for i in range(1, pred.size(1)):
                sym = TGT.vocab.itos[pred[0, i]]
                if sym == eos: break
                seq.append(sym)
            seq = tok_en.decode_pieces(seq)
            print(f"pred {pred_proba:.2f}:", seq)
                
        seq = []
        for i in range(1, batch.trg.size(1)):
            sym = TGT.vocab.itos[batch.trg[0, i]]
            if sym == eos: break
            seq.append(sym)
        seq = tok_en.decode_pieces(seq)
        print("Target:", seq)
        break


Source: правила реконструкции
Translation:
pred 17.67: that cost cost three short cost yet three
pred 17.61: that cost cost power gdp gdp gdp gdp
pred 17.33: that cost cost three short imf cost three
pred 17.28: that cost cost cost three short cost yet
pred 17.23: that cost cost power gdp gdp gdp yet
Target: the rules of reconstruction


In [0]:
len_test = len(list(iter(test_iter)))

In [0]:
pad = "<pad>"

In [0]:
hypotheses = []
references = []

model.eval()
with torch.no_grad():
    for batch in tqdm.tqdm_notebook(test_iter, total=len_test, desc=f"Test: ", leave=True):
        for j, el in enumerate(batch.src):
            src = batch.src[j:j+1]
            src_key_padding_mask = src != TGT.vocab.stoi['<pad>']
            beam = beam_search(model, src, src_key_padding_mask, max_len=10, k=10)
            for pred, pred_proba in beam[:1]:                
                seq = []
                for i in range(1, pred.size(1)):
                    sym = TGT.vocab.itos[pred[0, i]]
                    if sym == eos: break
                    seq.append(sym)
                seq = tok_en.decode_pieces(seq)
                trg = batch.trg[j:j+1].tolist()[0]
                ref = []
                for i, se in enumerate(trg):
                     tok = TGT.vocab.itos[trg[i]]
                     if tok == TGT.vocab.stoi['<pad>']: break
                     ref.append(tok)
                hypotheses.append(seq.split())
                references.append(tok_en.decode_pieces(ref).split())

In [0]:
a = bleu(references, hypotheses, 
     smoothing_function=translate.bleu_score.SmoothingFunction().method3,
     auto_reweigh=True)

print(a)

NameError: ignored

In [0]:
corpus_bleu(
    references, hypotheses, smoothing_function=SmoothingFunction().method3,
    auto_reweigh=True
)

4.20871424796906e-05

In [0]:
def beam_search(model, src, src_mask, max_len=10, k=5):
    <TODO>

In [0]:
model.eval()
with torch.no_grad():
    for i, batch in enumerate(valid_iter):
        src = batch.src[:1]
        src_key_padding_mask = src != SRC.vocab.stoi["<pad>"]
        beam = beam_search(model, src, src_key_padding_mask)
        
        seq = []
        for i in range(1, src.size(1)):
            sym = SRC.vocab.itos[src[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_ru.decode_pieces(seq)
        print("\nSource:", seq)
        
        print("Translation:")
        for pred, pred_proba in beam:                
            seq = []
            for i in range(1, pred.size(1)):
                sym = TGT.vocab.itos[pred[0, i]]
                if sym == "</s>": break
                seq.append(sym)
            seq = tok_en.decode_pieces(seq)
            print(f"pred {pred_proba:.2f}:", seq)
                
        seq = []
        for i in range(1, batch.tgt.size(1)):
            sym = TGT.vocab.itos[batch.tgt[0, i]]
            if sym == "</s>": break
            seq.append(sym)
        seq = tok_en.decode_pieces(seq)
        print("Target:", seq)
        break


Source: рост
Translation:
pred -1.31: growth
pred -2.03: growth growth
pred -3.63: rising growth
pred -3.89: growth in growth
pred -4.38: growth growth growth
Target: inflation


/home/denis.litvinov/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



In [0]:
from nltk.translate.bleu_score import corpus_bleu

In [0]:
hypotheses = []
references = []

model.eval()
with torch.no_grad():
    for batch in test_iter:
        <TODO>

In [0]:
corpus_bleu(references, hypotheses, 
            smoothing_function=translate.bleu_score.SmoothingFunction().method3,
            auto_reweigh=True
           )

0.22829332685417014